## IMPORTS

In [1]:
import pandas as pd
import sys
import numpy as np

sys.path.append('../../')
from utils import clustering_utils, annotation_info

In [2]:
annotations_df = pd.read_csv('../eggnog/eggnog_compiled_2024_may15.annotations', sep='\t')
annotations_df

,query,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs
0,YF00009131.t1,5911.EAR95232,0.000000e+00,2740.1,"2BZI6@1|root,2SMM4@2759|Eukaryota,3ZB0Z@5878|C...",5878|Ciliophora,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,YF00009133.t1,5911.EAR95233,2.900000e-271,894.2,"COG5159@1|root,KOG1463@2759|Eukaryota,3ZBI5@58...",5878|Ciliophora,O,PCI/PINT associated module,-,-,...,ko:K03036,"ko03050,ko05169,map03050,map05169",M00341,-,-,"ko00000,ko00001,ko00002,ko03051",-,-,-,PCI
2,YF00009135.t1,102107.XP_008240144.1,1.900000e-05,18.2,"KOG4757@1|root,KOG4757@2759|Eukaryota,37QBS@33...",35493|Streptophyta,S,Protection of telomeres protein,-,"GO:0000228,GO:0000723,GO:0000781,GO:0000782,GO...",...,-,-,-,-,-,-,-,-,-,"POT1,POT1PC"
3,YF00009136.t1,5888.CAK91573,1.400000e-18,56.9,"2EJ1D@1|root,2SPB2@2759|Eukaryota,3ZC13@5878|C...",5888.CAK91573|-,S,Forkhead associated domain,-,-,...,-,-,-,-,-,-,-,-,-,-
4,YF00009137.t1,5911.EAR95236,0.000000e+00,2276.6,"2C19C@1|root,2QPW2@2759|Eukaryota,3ZE2M@5878|C...",2759|Eukaryota,S,Viral A-type inclusion protein repeat containi...,dec-1,"GO:0000003,GO:0002064,GO:0002065,GO:0002066,GO...",...,ko:K17973,-,-,-,-,"ko00000,ko03029",-,-,-,"DEC-1_C,DEC-1_N,Dec-1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15725,YF00013871.t1,5911.EAR97259,9.400000e-187,616.0,"COG1028@1|root,KOG1205@2759|Eukaryota,3ZCYB@58...",5878|Ciliophora,Q,KR domain,-,-,...,-,-,-,-,-,-,-,-,-,adh_short
15726,YF00013873.t1,5911.EAR97257,5.600000e-77,244.7,"2F06U@1|root,2T1DW@2759|Eukaryota,3ZF56@5878|C...",5878|Ciliophora,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
15727,YF00036611.t1,4641.GSMUA_Achr10P19120_001,1.500000e-07,26.1,"KOG1864@1|root,KOG1864@2759|Eukaryota,37Q40@33...",35493|Streptophyta,O,Belongs to the peptidase C19 family,-,-,...,ko:K11842,-,-,-,-,"ko00000,ko01000,ko01002,ko04121",-,-,-,UCH
15728,YF00013878.t1,5911.EAR97253,0.000000e+00,4710.6,"2CBV6@1|root,2QQ4Q@2759|Eukaryota,3ZBD8@5878|C...",5878|Ciliophora,S,Encoded by,-,-,...,-,-,-,-,-,-,-,-,-,-


In [3]:
full_filtered_df = pd.read_csv('../../active_fastas/rna_seq.csv')
# full_filtered_df = full_filtered_df.rename(columns={'Unnamed: 0': 'TTHERM_ID'})
# full_filtered_norm_df = microarray_utils.normalize_expression_per_gene(full_filtered_df)

num_genes = full_filtered_df.shape[0]
num_genes

21524

In [4]:
full_filtered_df.sample(10)

,TTHERM_ID,000min,030min,060min,090min,120min,150min,180min,210min,240min
19075,YF00030960.t1,1.292252,1.571927,-0.558739,-0.446284,-0.803021,1.195372,-1.004082,-0.129851,-1.117575
14423,YF00022596.t1,1.043289,-1.964745,-1.035930,0.327206,1.158031,-0.343302,-0.574798,0.935366,0.454881
1038,YF00001420.t1,-0.663832,-0.775714,0.189484,2.067307,-0.294721,1.383456,-0.194415,-1.124664,-0.586901
21022,YF00037259.t1,1.933060,0.903684,0.283978,-0.395091,-0.027271,0.471956,-0.441953,-1.550065,-1.178298
17043,YF00026350.t1,-2.078636,-1.276654,-0.196192,0.531988,0.955431,0.900906,0.806783,0.558346,-0.201973
4923,YF00007126.t1,-1.020501,-0.034439,-0.009822,1.048781,1.194902,1.199828,0.337617,-1.052674,-1.663691
13097,YF00020358.t1,2.217286,0.786520,-0.046687,-0.377235,-0.556193,-0.570924,-1.020296,-1.096780,0.664310
10241,YF00016065.t1,1.996286,0.879412,0.602431,0.212137,-0.028787,-0.359030,-1.288489,-1.095303,-0.918657
7353,YF00011408.t1,0.285002,-0.224002,-1.385400,-0.377441,-0.939964,-0.728950,0.194446,1.194693,1.981616
5918,YF00008827.t1,-0.257926,-0.341534,-0.904295,-1.120536,-0.849117,-0.586613,1.004023,1.547245,1.508753


In [5]:
cycle_expression_conflict_df = full_filtered_df.loc[
    (full_filtered_df['000min'] > 0.9)
    &
    (full_filtered_df['210min'] < 0.1)
    &
    (full_filtered_df['240min'] < 0.1)
]

In [6]:
cycle_expression_conflict_df

,TTHERM_ID,000min,030min,060min,090min,120min,150min,180min,210min,240min
0,YF00000001.t1,0.910193,-0.286020,-0.119706,0.893590,0.928947,0.176211,0.727746,-1.052564,-2.178396
6,YF00000011.t1,2.261305,0.620949,-1.042034,-1.259590,0.252273,-0.102887,0.330185,-0.734130,-0.326072
8,YF00000013.t1,0.935421,0.699402,-0.142884,0.016602,0.136700,1.443761,0.185203,-1.394728,-1.879477
16,YF00000023.t1,1.351731,0.149180,0.587951,-1.639571,-0.157602,1.150480,0.598685,-0.615899,-1.424954
18,YF00000028.t1,2.259856,-0.422235,-0.963075,-0.670578,0.600565,0.554869,0.164753,-0.271998,-1.252156
...,...,...,...,...,...,...,...,...,...,...
21513,YF00038363.t1,2.481030,0.516379,-0.101797,-0.486988,-0.813020,0.099809,0.032080,-1.152030,-0.575464
21515,YF00038368.t1,1.369232,-0.645802,-1.089678,-0.064023,0.125732,2.039101,-0.137823,-0.955091,-0.641647
21518,YF00038372.t1,1.034074,0.597451,-1.152814,-1.240039,-0.707393,1.968434,0.126318,-0.557980,-0.068052
21521,YF00038377.t1,2.725986,-0.627384,-0.813547,-0.394835,-0.480004,-0.116765,0.062185,-0.025355,-0.330281


In [7]:
clustering_utils.compute_enrichment(cycle_expression_conflict_df, np.zeros(cycle_expression_conflict_df.shape[0]))

,module,term,info,fold_change,bonferroni
0,0.0,ko:K07818,perforin 1,3.551928,1.167158e-02
0,0.0,LRR_6,Leucine Rich repeat,1.891559,1.670589e-11
1,0.0,MACPF,MAC/Perforin domain,3.207427,1.366058e-02


In [8]:
filtered_annotations_df = annotations_df.loc[annotations_df['query'].isin(list(cycle_expression_conflict_df['TTHERM_ID'].values))]
filtered_annotations_df

,query,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs
3,YF00009136.t1,5888.CAK91573,1.400000e-18,56.9,"2EJ1D@1|root,2SPB2@2759|Eukaryota,3ZC13@5878|C...",5888.CAK91573|-,S,Forkhead associated domain,-,-,...,-,-,-,-,-,-,-,-,-,-
13,YF00036151.t1,5911.EAR86932,1.600000e-19,57.2,"2D4M2@1|root,2SVJ4@2759|Eukaryota,3ZEY9@5878|C...",5878|Ciliophora,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
17,YF00036152.t1,5911.EAR95256,0.000000e+00,2828.9,"KOG0498@1|root,KOG0500@1|root,KOG0498@2759|Euk...",5911.EAR95256|-,U,cyclic nucleotide-gated ion channel activity,-,-,...,-,-,-,-,-,-,-,-,-,-
18,YF00009162.t1,5911.EAR90913,3.300000e-14,47.6,"COG2202@1|root,KOG0501@2759|Eukaryota",5911.EAR90913|-,T,phosphorelay sensor kinase activity,-,-,...,-,-,-,-,-,-,-,-,-,-
32,YF00009184.t1,5888.CAK82239,1.000000e-58,189.1,"2EI24@1|root,2SNK1@2759|Eukaryota,3ZCY0@5878|C...",5878|Ciliophora,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15702,YF00013831.t1,5911.EAR97298,0.000000e+00,1658.9,"28KGW@1|root,2TMCI@2759|Eukaryota,3ZBVK@5878|C...",5878|Ciliophora,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
15710,YF00013849.t1,5911.EAR97282,3.100000e-252,830.8,"KOG3636@1|root,KOG3636@2759|Eukaryota,3ZEDW@58...",5878|Ciliophora,S,"Domain in Tre-2, BUB2p, and Cdc16p. Probable R...",-,-,...,-,-,-,-,-,-,-,-,-,RabGAP-TBC
15714,YF00013854.t1,5911.EAR97276,1.100000e-128,424.6,"COG4826@1|root,KOG2392@2759|Eukaryota",2759|Eukaryota,O,serine-type endopeptidase inhibitor activity,-,-,...,ko:K13963,"ko05146,map05146",-,-,-,"ko00000,ko00001",-,-,-,Serpin
15715,YF00013855.t1,3827.XP_004510833.1,9.100000e-39,129.1,"COG1100@1|root,KOG0098@2759|Eukaryota,37PZ3@33...",35493|Streptophyta,U,Ras-related protein,-,"GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",...,ko:K07877,"ko04152,map04152",-,-,-,"ko00000,ko00001,ko04031,ko04131,ko04147",-,-,-,Ras


In [9]:
terms_cols = ['COG_category', 'GOs', 'KEGG_ko', 'EC', 'PFAMs']

term_frequency = {}

for terms_col in terms_cols:
    for terms in filtered_annotations_df[terms_col].values:
        term_list = terms.split(',')
        for term in term_list:
            if term not in term_frequency:
                term_frequency[term] = 0
            term_frequency[term] += 1


In [10]:
sorted_term_frequency = dict(sorted(term_frequency.items(), key=lambda item: item[1], reverse=True))
sorted_term_frequency

{'-': 10661,
 'S': 815,
 'O': 297,
 'GO:0005575': 286,
 'GO:0008150': 282,
 'GO:0005623': 271,
 'GO:0044464': 271,
 'GO:0005622': 267,
 'GO:0044424': 267,
 'T': 255,
 'GO:0003674': 246,
 'GO:0009987': 239,
 'GO:0043226': 239,
 'GO:0005737': 237,
 'GO:0043229': 235,
 'GO:0043227': 224,
 'GO:0043231': 203,
 'GO:0044444': 195,
 'GO:0008152': 189,
 'GO:0044237': 183,
 'GO:0071704': 176,
 'GO:0044238': 172,
 'GO:0003824': 166,
 'GO:0044422': 165,
 'GO:0044446': 164,
 'GO:0065007': 161,
 'GO:0006807': 159,
 'GO:0043170': 148,
 'GO:0050789': 148,
 'GO:0005488': 145,
 'U': 140,
 'GO:0050794': 138,
 'Pkinase': 137,
 'GO:0050896': 135,
 'GO:0044260': 130,
 'GO:0071840': 130,
 'LRR_6': 130,
 'GO:0016020': 127,
 'GO:1901564': 127,
 'GO:0032502': 127,
 'GO:0016043': 125,
 'GO:0048856': 122,
 'GO:0051179': 117,
 'GO:0005634': 115,
 'GO:0032501': 114,
 'GO:0019538': 110,
 'GO:0044267': 106,
 'GO:0012505': 103,
 'GO:0005515': 103,
 'GO:0051234': 101,
 'GO:0007275': 100,
 'GO:0051716': 99,
 'GO:0043412

In [11]:
all_terms = []

all_counts = []

all_descs = []

for k, v in sorted_term_frequency.items():
    all_terms.append(k)
    all_counts.append(v)
    if k in annotation_info.go_df['GOs'].values:
       all_descs.append('; '.join(annotation_info.get_GO_info(k)))
       continue
    if k in annotation_info.kegg_df['KEGG_ko'].values:
       all_descs.append(annotation_info.get_KEGG_info(k))
       continue
    if k in annotation_info.ec_df['EC'].values:
       all_descs.append(annotation_info.get_EC_info(k))
       continue
    if k in annotation_info.pfam_df['PFAMs'].values:
       all_descs.append(annotation_info.get_PFAM_info(k))
       continue
    if k in annotation_info.COG_dict:
       all_descs.append(annotation_info.COG_dict[k])
       continue
    
    all_descs.append('-')

print(len(all_terms), len(all_counts), len(all_descs))

term_frequency_df = pd.DataFrame({
   'term': all_terms,
   'count': all_counts,
   'all_descs': all_descs
})

term_frequency_df.head(40)

7024 7024 7024


,term,count,all_descs
0,-,10661,-; -; -
1,S,815,Function unknown
2,O,297,"Posttranslational modification, protein turnov..."
3,GO:0005575,286,"cellular_component; A location, relative to ce..."
4,GO:0008150,282,biological_process; A biological process is th...
5,GO:0005623,271,obsolete cell; OBSOLETE. The basic structural ...
6,GO:0044464,271,obsolete cell part; OBSOLETE. Any constituent ...
7,GO:0005622,267,intracellular anatomical structure; A componen...
8,GO:0044424,267,obsolete intracellular part; OBSOLETE. Any con...
9,T,255,Signal transduction mechanisms
